In [1]:
import pandas as pd
import pyodbc
import numpy as np
import os
 
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sqlalchemy import create_engine

In [2]:
print("Available ODBC drivers:")
for driver in pyodbc.drivers():
    print(driver)
 
# Connection to the master database
SERVER = 'masterdatabase-apl.database.windows.net'
DATABASE = 'masterdata'
USERNAME = 'PowerBI'
PASSWORD = os.getenv('MDP_SQLServer_masterdata')
DRIVER = "ODBC Driver 17 for SQL Server"

#Créer une variable de chaîne de connexion à l’aide de l’interpolation de chaîne.
connectionString = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};Encrypt=yes;TrustServerCertificate=yes;'
conn = pyodbc.connect(connectionString)
cursor = conn.cursor()
 
print(f"Connexion établie au server : {SERVER} ")



Available ODBC drivers:
SQL Server
ODBC Driver 17 for SQL Server
SQL Server Native Client RDA 11.0
Connexion établie au server : masterdatabase-apl.database.windows.net 


### Clients double mission

In [8]:
# Récupérer les données
recup_data = """ 
SELECT  
    a.*, 
    c.Accountid, 
    b.[Type]
FROM [dbo].[Missions] a 
INNER JOIN [dbo].[MissionTypes] b ON b.ID = a.FkIdTypeMission
INNER JOIN [dbo].[TempsDimAccounts] c ON c.ID = a.FkIdAccount
WHERE b.[Type] = 'MIS'
"""

df = pd.read_sql(recup_data, conn)


# Étape 1 : Compter le nombre de missions PAR CLIENT ET PAR ANNÉE
missions_par_client_annee = df.groupby(["Accountid", "Annee"]).size().reset_index(name="NombreMissions")

# Étape 2 : Ne garder que ceux qui ont 2 missions ou plus dans la même année
clients_annee_multiples = missions_par_client_annee[missions_par_client_annee["NombreMissions"] >= 2]

# Étape 3 : Merge pour ne garder que les missions de ces cas-là
df_clients_multiples_annee = df.merge(clients_annee_multiples, on=["Accountid", "Annee"])

# Étape 4 : Trier par Année, Accountid, puis ID
df_clients_multiples_annee = df_clients_multiples_annee.sort_values(by=["Annee", "Accountid", "ID"])

# Affichage
df_clients_multiples_annee.head(500)


C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\4061446282.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(recup_data, conn)


,ID,Annee,MoisCloture,BudgetEnHeures,BudgetEnEuros,IDOpportunite,CrmName,RefDate,AutoCloseDate,FkIdTypeMission,DenormalizedNoClient,FkIdAccount,AccountId,MergeKey,Accountid,Type,NombreMissions
194,11203,2004,12,0.00,1800.0,00000000-0000-0000-0000-000000000000,MIS-2004-12-01,2004-12-31,2005-05-15,1,01459,8518,None,120041201459,236F224C-69FE-4040-AB30-844E48F4BEB5,MIS,2
199,11389,2004,6,0.00,1800.0,00000000-0000-0000-0000-000000000000,MIS-2004-06-01,2004-06-30,2005-05-15,1,01459,8518,None,12004601459,236F224C-69FE-4040-AB30-844E48F4BEB5,MIS,2
196,11250,2004,12,41.00,3500.0,00000000-0000-0000-0000-000000000000,MIS-2004-12-01,2004-12-31,2004-07-09,1,01529,8566,None,120041201529,AD869361-1254-4F27-AC00-A20AD962BB8F,MIS,2
200,11491,2004,9,85.00,6000.0,00000000-0000-0000-0000-000000000000,MIS-2004-09-01,2004-09-30,2005-10-07,1,01529,8566,None,12004901529,AD869361-1254-4F27-AC00-A20AD962BB8F,MIS,2
193,11191,2004,12,25.00,1700.0,00000000-0000-0000-0000-000000000000,MIS-2004-12-01,2004-12-31,2005-10-06,1,01430,1491,None,120041201430,F922C68A-B4E7-EB11-BACB-0022489B9951,MIS,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,2495,2024,8,42.45,4440.0,00000000-0000-0000-0000-000000000000,LM 31/03/2024,2024-08-31,2024-11-30,1,11534,1038,None,12024811534,E2C43B1C-877C-ED11-81AD-000D3ADF7AEC,MIS,2
536,37964,2024,3,44.00,4440.0,00000000-0000-0000-0000-000000000000,MIS-2024-03-01,2024-03-31,2023-04-30,1,11534,1038,None,12024311534,E2C43B1C-877C-ED11-81AD-000D3ADF7AEC,MIS,2
112,7509,2024,3,43.45,6950.0,E912935C-E27C-4681-AA3F-F5DFC832306E,LM 31/03/2024,2024-03-31,2024-10-31,1,11310,1123,None,120241111310,E3CC5F40-073F-ED11-9DB0-000D3ADF7AEC,MIS,2
557,38014,2024,6,75.00,6950.0,00000000-0000-0000-0000-000000000000,MIS-2024-06-01,2024-06-30,2099-12-31,1,11310,1123,None,12024611310,E3CC5F40-073F-ED11-9DB0-000D3ADF7AEC,MIS,2


### Fusion des clients

In [9]:

recup_data = """ 
SELECT  
    a.*, 
    c.Accountid, 
    b.[Type]
FROM [dbo].[Missions] a 
INNER JOIN [dbo].[MissionTypes] b ON b.ID = a.FkIdTypeMission
INNER JOIN [dbo].[TempsDimAccounts] c ON c.ID = a.FkIdAccount
WHERE b.[Type] = 'MIS'
"""

df = pd.read_sql(recup_data, conn)

missions_par_client_annee = df.groupby(["Accountid", "Annee"]).size().reset_index(name="NombreMissions")
clients_annee_multiples = missions_par_client_annee[missions_par_client_annee["NombreMissions"] >= 2]
df_clients_multiples_annee = df.merge(clients_annee_multiples, on=["Accountid", "Annee"])

#  Grouper et sommer BudgetEnHeures et BudgetEnEuros
df_somme = df_clients_multiples_annee.groupby(["Accountid", "Annee"]).agg({
    "BudgetEnHeures": "sum",
    "BudgetEnEuros": "sum"
}).reset_index()


# (on prend arbitrairement la première mission de chaque groupe)
colonnes_info = [
    "Accountid", "Annee", "ID", "MoisCloture", "IDOpportunite", "CrmName",
    "RefDate", "AutoCloseDate", "FkIdTypeMission", "DenormalizedNoClient",
    "FkIdAccount", "MergeKey", "Type"
]

df_info = df_clients_multiples_annee.sort_values(by=["Annee", "Accountid", "ID"]).drop_duplicates(subset=["Accountid", "Annee"])[colonnes_info]

# Fusionner les infos avec les budgets sommés
df_fusion = pd.merge(df_info, df_somme, on=["Accountid", "Annee"])

# Résultat final
df_fusion = df_fusion.sort_values(by=["Annee", "Accountid"])


df_fusion.head(500)


C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\492389260.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(recup_data, conn)


,Accountid,Annee,ID,MoisCloture,IDOpportunite,CrmName,RefDate,AutoCloseDate,FkIdTypeMission,DenormalizedNoClient,FkIdAccount,MergeKey,Type,BudgetEnHeures,BudgetEnEuros
0,236F224C-69FE-4040-AB30-844E48F4BEB5,2004,11203,12,00000000-0000-0000-0000-000000000000,MIS-2004-12-01,2004-12-31,2005-05-15,1,01459,8518,120041201459,MIS,0.00,3600.0
1,AD869361-1254-4F27-AC00-A20AD962BB8F,2004,11250,12,00000000-0000-0000-0000-000000000000,MIS-2004-12-01,2004-12-31,2004-07-09,1,01529,8566,120041201529,MIS,126.00,9500.0
2,F922C68A-B4E7-EB11-BACB-0022489B9951,2004,11191,12,00000000-0000-0000-0000-000000000000,MIS-2004-12-01,2004-12-31,2005-10-06,1,01430,1491,120041201430,MIS,29.00,3300.0
3,F9618408-B5E7-EB11-BACB-0022489B9F4E,2004,11249,12,00000000-0000-0000-0000-000000000000,MIS-2004-12-01,2004-12-31,2005-04-30,1,01528,4539,120041201528,MIS,8.00,630.0
4,00B31C52-B6E7-EB11-BACB-0022489B9951,2006,12063,11,00000000-0000-0000-0000-000000000000,MIS-2006-11-01,2006-11-30,2007-03-31,1,01828,1533,120061101828,MIS,0.00,1400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,FBA47CEA-839D-ED11-AAD1-000D3ADF7AEC,2025,8320,9,2E0AE0AE-4956-4C39-89BD-4FBECD33667D,Lettre de mission initiale 2023,2025-09-30,2026-04-30,1,11684,1202,120251011684,MIS,17.90,3418.0
307,FD42CB56-4A38-458D-91B7-619D3887D229,2025,8916,9,D88535C6-43B9-4E4D-BDC6-847F083983A8,Lettre de mission 30/09/25,2025-09-30,2026-04-30,1,09000,7998,120251209000,MIS,11.66,2316.0
308,FDF573CE-24E8-EB11-BACB-0022489B9951,2025,1430,9,00000000-0000-0000-0000-000000000000,LDM 2022,2025-09-30,2026-04-30,1,08969,4404,12025908969,MIS,58.75,9682.0
309,FF7BD052-FCE7-EB11-BACB-0022489B9F4E,2025,8631,12,8F55C35A-CBFD-EF11-BAE3-000D3ADDB2ED,MIS-2025-12,2025-12-31,2026-07-31,1,07305,6659,120251207305,MIS,13.00,2881.0


In [ ]:
#  lister les tables disponibles
query_tables = """
SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE'
"""

df_tables = pd.read_sql(query_tables, conn)
df_tables.head(50)


C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\311715497.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(query_tables, conn)


,TABLE_SCHEMA,TABLE_NAME
0,dbo,MaintiensMissions
1,dbo,TempGuids
2,dbo,MissionTypes
3,dbo,ProcessExecutionSummary
4,dbo,Missions
5,dbo,FecLines
6,dbo,AccountsHistory
7,dbo,QuestionnaireInvitations
8,dbo,CrmArticlesOpportunite
9,dbo,CrmProduitsOpportunite


In [18]:
query_columns = """
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = 'Temps' AND TABLE_NAME = 'SaisieTemps'
"""

df_colonnes = pd.read_sql(query_columns, conn)
df_colonnes


C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\1170496455.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_colonnes = pd.read_sql(query_columns, conn)


,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,...,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME


In [26]:
query = """
SELECT TOP 1000 
t.*,
a.code 
FROM [dbo].[SaisieTemps] as t
INNER JOIN [dbo].[TempsDimArticles] a ON a.ID = t.FkIdArticleCatalogue	
"""

df_saisie_temps = pd.read_sql(query, conn)
df_saisie_temps.head()


C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\1535337796.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_saisie_temps = pd.read_sql(query, conn)


,ID,Temps,FkIdArticleCatalogue,DateSaisie,Date,Description,FkIdMission,Locked,CoutEnEuros,FkIdCollaborateur,Tag,code
0,1609,0.75,7,2024-10-07 08:05:21.453,2024-09-17,contrôle inqom,3878,True,65.46,22,NaN,5
1,1610,0.50,7,2024-09-29 15:05:09.720,2024-09-17,contrôle inqom,3563,True,43.64,22,NaN,5
2,1676,0.75,2,2024-09-18 14:53:46.100,2024-09-18,pointage fournisseur Nao,3674,True,65.46,76,NaN,3
3,1686,0.25,1,2024-09-18 15:22:21.107,2024-09-18,cloture et edi,3252,True,19.79,60,NaN,1
4,1715,1.00,2,2024-09-29 15:08:33.800,2024-09-19,point compta,2699,True,87.28,22,NaN,3


In [28]:
# 1. Charger la table Missions avec client et année
query_missions = """
SELECT ID, FkIdAccount, Annee
FROM [dbo].[Missions]
"""
df_missions = pd.read_sql(query_missions, conn)
print("Missions loaded:", df_missions.shape)
print(df_missions.head())

# 2. Fusionner les temps saisis avec les missions
df_saisie_avec_mission = df_saisie_temps.merge(
    df_missions,
    left_on="FkIdMission",
    right_on="ID",
    suffixes=("", "_Mission")
)
print("Après fusion saisie + missions:", df_saisie_avec_mission.shape)
print(df_saisie_avec_mission.head())

# 3. Grouper par client, code, année et sommer le temps
df_regroupement = df_saisie_avec_mission.groupby(
    ["FkIdAccount", "code", "Annee"]
)["Temps"].sum().reset_index(name="TotalTemps")
print("Après groupement et sommation:")
print(df_regroupement.head())


Missions loaded: (34025, 3)
   ID  FkIdAccount  Annee
0   3         8070   2024
1   6           11   2024
2   9           30   2025
3  13         2191   2025
4  27         6892   2025
Après fusion saisie + missions: (1000, 15)
     ID  Temps  FkIdArticleCatalogue              DateSaisie        Date  \
0  1609   0.75                     7 2024-10-07 08:05:21.453  2024-09-17   
1  1610   0.50                     7 2024-09-29 15:05:09.720  2024-09-17   
2  1676   0.75                     2 2024-09-18 14:53:46.100  2024-09-18   
3  1686   0.25                     1 2024-09-18 15:22:21.107  2024-09-18   
4  1715   1.00                     2 2024-09-29 15:08:33.800  2024-09-19   

                Description  FkIdMission  Locked  CoutEnEuros  \
0            contrôle inqom         3878    True        65.46   
1            contrôle inqom         3563    True        43.64   
2  pointage fournisseur Nao         3674    True        65.46   
3            cloture et edi         3252    True        

C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\866748577.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_missions = pd.read_sql(query_missions, conn)


In [29]:
# Supposons que tu as déjà df_saisie_avec_mission qui contient : FkIdAccount, code, Annee, Temps

# Filtrer uniquement les codes 1, 3, 4, 5 (en adaptant selon le type exact de 'code')
codes_a_garder = ['1', '3', '4', '5']  # si ce sont des strings
# Si ce sont des entiers, enlève les quotes : [1,3,4,5]

df_filtre = df_saisie_avec_mission[df_saisie_avec_mission['code'].astype(str).isin(codes_a_garder)]

# Grouper et sommer les temps
df_grouped = df_filtre.groupby(
    ['Annee', 'FkIdAccount', 'code']
)['Temps'].sum().reset_index(name='TotalTemps')

# Trier par Année, Client, code
df_sorted = df_grouped.sort_values(by=['Annee', 'FkIdAccount', 'code'])

df_sorted.head(100)  # affiche les 100 premiers résultats


,Annee,FkIdAccount,code,TotalTemps
0,2024,133,3,0.40
1,2024,134,3,0.50
2,2024,139,5,2.25
3,2024,163,3,0.15
4,2024,173,5,1.25
...,...,...,...,...
95,2024,4789,3,0.10
96,2024,4852,3,0.25
97,2024,4852,5,2.50
98,2024,4971,3,0.35


In [30]:
import pandas as pd

# 1. Charger les missions (ID, client, année)
query_missions = """
SELECT ID, FkIdAccount, Annee
FROM [dbo].[Missions]
"""
df_missions = pd.read_sql(query_missions, conn)

# 2. Charger les saisies temps avec code article
query_saisie = """
SELECT 
    t.ID, t.Temps, t.FkIdArticleCatalogue, t.DateSaisie, t.Description, 
    t.FkIdMission, t.FkIdCollaborateur, a.code
FROM [dbo].[SaisieTemps] t
INNER JOIN [dbo].[TempsDimArticles] a ON a.ID = t.FkIdArticleCatalogue
"""
df_saisie_temps = pd.read_sql(query_saisie, conn)

# 3. Fusionner saisies avec missions pour récupérer client et année
df_saisie_avec_mission = df_saisie_temps.merge(
    df_missions,
    left_on="FkIdMission",
    right_on="ID",
    suffixes=("", "_Mission")
)

# 4. Filtrer les codes qui t’intéressent
codes_a_garder = ['1', '3', '4', '5']
df_filtre = df_saisie_avec_mission[df_saisie_avec_mission['code'].astype(str).isin(codes_a_garder)]

# 5. Grouper par client, code, année pour sommer le temps et concaténer les IDs des missions
df_result = df_filtre.groupby(
    ['FkIdAccount', 'code', 'Annee']
).agg(
    TotalTemps=('Temps', 'sum'),
    ListeIDsMissions=('FkIdMission', lambda x: ','.join(map(str, sorted(x.unique()))))
).reset_index()

# 6. Trier
df_result = df_result.sort_values(by=['Annee', 'FkIdAccount', 'code'])

# 7. Afficher
print(df_result.head(100))


C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\2121312503.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_missions = pd.read_sql(query_missions, conn)
C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\2121312503.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_saisie_temps = pd.read_sql(query_saisie, conn)


       FkIdAccount code  Annee  TotalTemps ListeIDsMissions
10020         2739    3   2014        1.65            41996
23084         4965    1   2015       34.80            21849
23085         4965    3   2015        0.35            21849
23086         4965    3   2016        0.70            23284
23093         4965    4   2016        2.00            23284
...            ...  ...    ...         ...              ...
1326           366    3   2019        0.66            28881
1335           366    5   2019        0.50            28881
1339           367    1   2019        5.83            28882
1345           367    3   2019        0.37            28882
1353           367    5   2019        1.50            28882

[100 rows x 5 columns]


In [32]:
recup_data_console = """
WITH TEMPS_DATA AS (
    SELECT
        t.FkIdMission,
        a.code,
        t.Temps,
        t.CoutEnEuros as PR,
        ROUND(1.5 * t.CoutEnEuros , 2) AS PV
 
    FROM SaisieTemps t
    LEFT JOIN TempsDimArticles a ON a.ID = t.FkIdArticleCatalogue
    LEFT JOIN Missions m ON t.FkIdMission = m.ID
    WHERE t.Temps <> 0
      AND m.[AutoCloseDate] < CAST(GETDATE() AS DATE) -- Missions closed before today
      AND m.[FkIdTypeMission] = 1 -- mission régulière
      AND a.Code IN ('1', '3', '4', '5', '6') -- Codes d'articles de base
      AND m.[DenormalizedNoClient] NOT IN (
            '', '000000', '99044', '99049', '99072', '04647',
            '04976', '12143', 'DEVCALL', '06141', 'DECVALL',
            '05848', '06484', '9AGEFOSO'
      )
)
 
SELECT
    m.[ID],
    m.[CrmName],
    m.[RefDate],
    m.[AutoCloseDate],
    m.[DenormalizedNoClient] AS NoClient,
 
    CONCAT(YEAR(m.[RefDate]), m.[DenormalizedNoClient]) AS composite_key,
 
    SUM(CASE WHEN d.code = '1' THEN d.temps ELSE 0 END) AS temps_code1,
    SUM(CASE WHEN d.code = '3' THEN d.temps ELSE 0 END) AS temps_code3,
    SUM(CASE WHEN d.code = '4' THEN d.temps ELSE 0 END) AS temps_code4,
    SUM(CASE WHEN d.code = '5' THEN d.temps ELSE 0 END) AS temps_code5,
    SUM(CASE WHEN d.code = '6' THEN d.temps ELSE 0 END) AS temps_code6,
    SUM(d.temps) AS temps_total,
 
    SUM(CASE WHEN d.code = '1' then d.PV else 0 end) as pv_code1,
    SUM(CASE WHEN d.code = '3' then d.PV else 0 end) as pv_code3,
    SUM(CASE WHEN d.code = '4' then d.PV else 0 end) as pv_code4,
    SUM(CASE WHEN d.code = '5' then d.PV else 0 end) as pv_code5,
    SUM(CASE WHEN d.code = '6' then d.PV else 0 end) as pv_code6,
    SUM(d.PV) AS pv_total
 
FROM Missions m
INNER JOIN TEMPS_DATA d ON m.ID = d.FkIdMission
 
GROUP BY
    m.[ID], m.[CrmName], m.[RefDate], m.[AutoCloseDate], m.[DenormalizedNoClient];
 
"""
 
Missions_console = pd.read_sql(recup_data_console, conn)
 
print(Missions_console.shape)
print(Missions_console.composite_key.duplicated().sum())
 
Missions_console = Missions_console.groupby('composite_key').agg({
    'ID':'max',
    'CrmName': 'first',
    'RefDate':'first',
    'AutoCloseDate':'max',
    'NoClient':'first',
    'temps_code1':'sum',
    'temps_code3':'sum',
    'temps_code4':'sum',
    'temps_code5':'sum',
    'temps_code6':'sum',
    'temps_total':'sum',
    'pv_code1':'sum',
    'pv_code3':'sum',
    'pv_code4':'sum',
    'pv_code5':'sum',
    'pv_code6':'sum',
    'pv_total':'sum'
    }).reset_index()
 
print(Missions_console.composite_key.duplicated().sum())
 
#Créer des colonnes binaires pour les codes d'articles si il sont présents ou pas dans la missions
for col in Missions_console.columns:
    if 'temps' in col:
        Missions_console[f"{col.split("_")[1]}"] =( Missions_console[col] > 0).astype(int)
 
print(Missions_console.shape)
 
 
Missions_console['rank_refdate_client'] = Missions_console.groupby('NoClient')['RefDate'].rank(method='dense', ascending=True).astype(int)
#Ne pas prendre en compte la première année d'exercice du client
Missions_console = Missions_console[Missions_console['rank_refdate_client']>=2]
#Supprimer la colonne rank_refdate_client
Missions_console = Missions_console[['ID', 'CrmName', 'RefDate', 'AutoCloseDate', 'NoClient',
       'composite_key', 'temps_code1', 'temps_code3', 'temps_code4',
       'temps_code5', 'temps_code6', 'temps_total', 'pv_code1', 'pv_code3',
       'pv_code4', 'pv_code5', 'pv_code6', 'pv_total', 'code1', 'code3',
       'code4', 'code5', 'code6']]
 
Missions_console.head(50)
 

C:\Users\Z35\AppData\Local\Temp\ipykernel_11584\959472117.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Missions_console = pd.read_sql(recup_data_console, conn)


(10001, 18)
79
0
(9922, 24)


,ID,CrmName,RefDate,AutoCloseDate,NoClient,composite_key,temps_code1,temps_code3,temps_code4,temps_code5,...,pv_code3,pv_code4,pv_code5,pv_code6,pv_total,code1,code3,code4,code5,code6
1,23284,MIS-2016-12-01,2016-12-31,2023-11-30,03617,201603617,0.00,0.70,2.00,0.00,...,85.92,231.28,0.00,0.00,317.20,0,1,1,0,0
6,24786,MIS-2017-12-01,2017-12-31,2023-11-30,03617,201703617,0.00,0.35,0.00,0.00,...,44.21,0.00,0.00,0.00,44.21,0,1,0,0,0
7,25010,MIS-2017-12-01,2017-12-31,2024-06-30,05185,201705185,0.00,1.35,0.00,0.00,...,159.26,0.00,0.00,0.00,159.26,0,1,0,0,0
19,26555,MIS-2018-12-01,2018-12-31,2020-05-27,05103,201805103,0.00,1.00,0.00,0.00,...,171.77,0.00,0.00,0.00,171.77,0,1,0,0,0
174,28006,MIS-2019-12-01,2019-12-31,2020-12-08,02631,201902631,5.75,1.92,0.00,1.50,...,165.18,0.00,93.33,0.00,744.70,1,1,0,1,0
188,28021,MIS-2019-12-01,2019-12-31,2020-09-30,02935,201902935,2.50,0.25,0.00,0.50,...,30.06,0.00,49.44,0.00,326.67,1,1,0,1,0
236,28064,MIS-2019-12-01,2019-12-31,2023-11-30,03617,201903617,0.00,0.25,0.00,0.00,...,30.06,0.00,0.00,184.61,214.67,0,1,0,0,1
281,29416,MIS-2019-09-01,2019-09-30,2021-04-30,03994,201903994,14.85,3.25,0.00,13.55,...,393.80,0.00,908.52,0.00,2706.39,1,1,0,1,0
330,28981,MIS-2019-03-01,2019-03-31,2020-06-30,04415,201904415,16.95,1.80,0.00,0.50,...,181.88,0.00,43.46,0.00,2178.18,1,1,0,1,0
363,28162,MIS-2019-12-01,2019-12-31,2022-02-28,04668,201904668,0.05,0.50,0.00,0.00,...,95.01,0.00,0.00,30.00,130.26,1,1,0,0,1


In [36]:
# Recalcul de l’année
Missions_console['Annee'] = pd.to_datetime(Missions_console['RefDate']).dt.year

# Regroupement
resultats = Missions_console.groupby(['NoClient', 'Annee']).agg({
    'ID': 'count',
    'temps_code1': 'sum',
    'temps_code3': 'sum',
    'temps_code4': 'sum',
    'temps_code5': 'sum',
    'temps_code6': 'sum',
    'temps_total': 'sum',
    'pv_code1': 'sum',
    'pv_code3': 'sum',
    'pv_code4': 'sum',
    'pv_code5': 'sum',
    'pv_code6': 'sum',
    'pv_total': 'sum'
}).reset_index()

# Filtrer uniquement les cas où il y a eu au moins 2 missions
resultats = resultats[resultats['ID'] >= 2]

# Renommer "ID" en "NbMissions"
resultats.rename(columns={'ID': 'NbMissions'}, inplace=True)

# Affichage
print(resultats.sort_values(['Annee', 'NoClient']).head(20))


Empty DataFrame
Columns: [NoClient, Annee, NbMissions, temps_code1, temps_code3, temps_code4, temps_code5, temps_code6, temps_total, pv_code1, pv_code3, pv_code4, pv_code5, pv_code6, pv_total]
Index: []
